In [1]:
%run functions.ipynb

Ready: 2023-05-03 00:39:52


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()
spark

In [250]:
def make_images(file_name, city, header_color, row_color, cnt=9999, size_x=17.5, size_y=10.8, font_size=18, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    df = pd.read_excel(file_name).drop(['tx_1','tx_2','tx_3','tx_4','tx_5'], axis=1)
    word_file = f'{file_name[:-5]}_report_{dt_now}.docx'
    print(word_file)
    
    df.columns = ['Рубрика', 'Поисковый запрос', 'Количество\nзапросов 2020',
           'Количество\nзапросов 2021', 'Количество\nзапросов 2022',
           'Количество\nзапросов 2023', 'Количество\nзапросов, итого',
           'Рейтинг\nпо запросам, итого', 'Рейтинг рубрики']

    for i in df.columns[2:-2]:
        df[i] = df[i].apply(make_sep)
        
    dups = df['Поисковый запрос'][(df['Поисковый запрос'].duplicated())&(df['Рейтинг рубрики'] > 0)].values 
    df_text = pd.read_excel(file_name, usecols=['Рубрика', 'Рейтинг рубрики', 'Поисковый запрос', 'tx_1'])
    df_text = df_text[df_text['tx_1'].notna()]
    tx_1 = df_text[df_text['Поисковый запрос'].apply(lambda x: x not in dups)][['tx_1', 'Рейтинг рубрики']]
    
    for i in tqdm(df['Рейтинг рубрики'].unique()[1:cnt+1]):  
        dfx = df[df['Рейтинг рубрики'] == i].drop('Рубрика', axis=1).iloc[:20]
        rubric = df[df['Рейтинг рубрики'] == i]['Рубрика'].iloc[0]
        col_name = f'Поисковые запросы по рубрике\n{rubric}'
        img_file_name = file_name.replace('files/','')[:-5]
        img_file_name_final = f'pictures/{img_file_name}_{i:02}.png'
        dfx = dfx.rename({'Поисковый запрос':col_name}, axis=1).drop(['Рейтинг рубрики', 'Рейтинг\nпо запросам, итого'], axis=1)
        words = tx_1[tx_1['Рейтинг рубрики'] == i]['tx_1'].values
        
        render_mpl_table(dfx, img_file_name_final, font_size=19, header_color=header_color, row_colors=[row_color, 'w'], dups=dups)
        if blank:
            try:
                doc = docx.Document(word_file)
            except:
                doc = docx.Document('files/report_ten_city.docx')
                doc.save(word_file)
            word=''
            doc.add_paragraph(word)
            doc.save(word_file)
        else:
            add_links(word_file, rubric, words, city)
        add_image_request(word_file, img_file_name_final, img_w=img_w, img_h=img_h, word_sample=word_sample)
        
make_requests_word = make_images

In [251]:
def render_mpl_table(data, file_name, index_col_to_colorize=5, dups=None, col_width=1.0, row_height=0.625, font_size=28,
                     header_color='#C0504D', row_colors=['#F2DCDB', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array([6.3*4, 3.94*4]))
        fig, ax = plt.subplots(figsize=size, dpi=2**8)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, colLoc='center', loc='bottom')

    mpl_table.auto_set_font_size(False)
    cells_dict = dict(six.iteritems(mpl_table._cells))
    
    im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\01_Logo_HSE_full_rus_PANTONE_for_dark_2.png")
    ax.figure.figimage(im, 85, 2800, zorder=1, alpha=1)
    
    
    fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
    #from colour import Color
    #colors = [i.get_hex() for i in list(Color("red").range_to(Color("green"),data.iloc[:20].shape[0]))]
    color_list_full = ['#F96A6C', '#F96B6C', '#F96C6C', '#F96D6C', '#F96E6C', '#F96F6D', '#F9706D', '#F9716D', '#F9726D', '#F9736D', '#F9746E', '#F9756E', '#F9766E', '#F9776E', '#F9786E', '#F9796E', '#F9796F', '#F97A6F'
                       , '#F97B6F', '#F97C6F', '#FA7C6F', '#FA7D6F', '#FA7E6F', '#FA7E70', '#FA7F70', '#FA8070', '#FA8170', '#FA8270', '#FA8370', '#FA8471', '#FA8571', '#FA8671', '#FA8771', '#FA8871', '#FA8972', '#FA8A72'
                       , '#FA8B72', '#FA8C72', '#FA8D72', '#FA8E73', '#FB8F73', '#FB9073', '#FB9173', '#FB9273', '#FB9373', '#FB9374', '#FB9474', '#FB9574', '#FB9674', '#FB9774', '#FB9874', '#FB9875', '#FB9975', '#FB9A75'
                       , '#FB9B75', '#FB9C75', '#FB9D75', '#FB9E76', '#FB9F76', '#FBA076', '#FBA176', '#FCA176', '#FCA276', '#FCA377', '#FCA477', '#FCA577', '#FCA677', '#FCA777', '#FCA877', '#FCA878', '#FCA978', '#FCAA78'
                       , '#FCAB78', '#FCAC78', '#FCAD78', '#FCAD79', '#FCAE79', '#FCAF79', '#FCB079', '#FCB179', '#FCB279', '#FCB27A', '#FCB37A', '#FDB47A', '#FDB57A', '#FDB67A', '#FDB77A', '#FDB87B', '#FDB97B', '#FDBA7B'
                       , '#FDBB7B', '#FDBC7B', '#FDBD7C', '#FDBE7C', '#FDBF7C', '#FDC07C', '#FDC17C', '#FDC27C', '#FDC27D', '#FDC37D', '#FDC47D', '#FDC57D', '#FDC67D', '#FEC77D', '#FEC77E', '#FEC87E', '#FEC97E', '#FECA7E'
                       , '#FECB7E', '#FECC7E', '#FECC7F', '#FECD7F', '#FECE7F', '#FECF7F', '#FED07F', '#FED17F', '#FED280', '#FED380', '#FED480', '#FED580', '#FED680', '#FED781', '#FED881', '#FFD981', '#FFDA81', '#FFDB81'
                       , '#FFDC81', '#FFDC82', '#FFDD82', '#FFDE82', '#FFDF82', '#FFE082', '#FFE182', '#FFE183', '#FFE283', '#FFE383', '#FFE483', '#FFE583', '#FFE683', '#FFE784', '#FFE884', '#FFE984', '#FFEA84', '#FFEB84'
                       , '#FEEA83', '#FDEA83', '#FCEA83', '#FBEA83', '#FAE983', '#F9E983', '#F8E983', '#F7E883', '#F6E883', '#F5E883', '#F4E883', '#F4E783', '#F3E783', '#F2E783', '#F1E783', '#F0E683', '#EFE683', '#EEE683'
                       ,'#EDE683', '#EDE582', '#ECE582', '#EBE582', '#EAE582', '#EAE482', '#E9E482', '#E8E482', '#E7E482', '#E6E482', '#E6E382', '#E5E382', '#E4E382', '#E3E382', '#E3E282', '#E2E282', '#E1E282', '#E0E282'
                       , '#DFE182', '#DEE182', '#DDE182', '#DCE182', '#DCE081', '#DBE081', '#DAE081', '#D9E081', '#D8DF81', '#D7DF81', '#D6DF81', '#D5DF81', '#D4DE81', '#D3DE81', '#D2DE81', '#D1DD81', '#D0DD81', '#CFDD81'
                       , '#CEDD81', '#CDDC81', '#CCDC81', '#CBDC81', '#CADB80', '#C9DB80', '#C8DB80', '#C7DB80', '#C7DA80', '#C6DA80', '#C5DA80', '#C4DA80', '#C3D980', '#C2D980', '#C1D980', '#C0D980', '#C0D880', '#BFD880'
                       , '#BED880', '#BDD880', '#BDD780', '#BCD780', '#BBD780', '#BAD780', '#B9D780', '#B9D67F', '#B8D67F', '#B7D67F', '#B6D67F', '#B6D57F', '#B5D57F', '#B4D57F', '#B3D57F', '#B2D47F', '#B1D47F', '#B0D47F'
                       , '#AFD47F', '#AED37F', '#ADD37F', '#ACD37F', '#ABD27F', '#AAD27F', '#A9D27F', '#A8D27F', '#A7D17E', '#A6D17E', '#A5D17E', '#A4D07E', '#A3D07E', '#A2D07E', '#A1D07E', '#A0CF7E', '#9FCF7E', '#9ECF7E'
                       , '#9DCE7E', '#9CCE7E', '#9BCE7E', '#9ACE7E', '#9ACD7E', '#99CD7E', '#98CD7E', '#97CD7E', '#96CC7D', '#95CC7D', '#94CC7D', '#93CC7D', '#93CB7D', '#92CB7D', '#91CB7D', '#90CB7D', '#90CA7D', '#8FCA7D'
                       , '#8ECA7D', '#8DCA7D', '#8CCA7D', '#8CC97D', '#8BC97D', '#8AC97D', '#89C97D', '#88C87D', '#87C87D', '#86C87D', '#85C77C', '#84C77C', '#83C77C', '#82C77C', '#81C67C', '#80C67C', '#7FC67C', '#7EC57C'
                       , '#7DC57C', '#7CC57C', '#7BC57C', '#7AC47C', '#79C47C', '#78C47C', '#77C37C', '#76C37C', '#75C37C', '#74C37C', '#73C27B'
                       , '#72C27B', '#71C27B', '#70C17B', '#6FC17B', '#6EC17B', '#6DC17B', '#6DC07B', '#6CC07B', '#6BC07B', '#6AC07B', '#69BF7B', '#68BF7B', '#67BF7B', '#66BF7B', '#66BE7B', '#65BE7B', '#64BE7B', '#63BE7B']
    colors = ([color_list_full[i] for i in range(0, len(color_list_full), len(color_list_full) % 20)] + [color_list_full[-1]])
    
    for k in sorted(cells_dict.keys()): #k[0] - строка, k[1] - столбец
        cell = cells_dict[k]
        cell.set_edgecolor(edge_color)
        row_index = k[0]
        column_index = k[1]
        
        # Высота и цвет заголовков
        if row_index == 0:
            rotation='horizontal' if k == (0,0) else 'vertical' 
            cell.set_text_props(color='w', rotation=rotation, ha='center')
            cell.set_facecolor(header_color)
            cell.set_height(0.07)
            
        # Высота и цвет остальных строк
        else:
            ha = 'right' if k[1] !=0 else 'left'
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
            cell.set_height(0.008)
            cell.set_text_props(wrap=True, ha=ha)
        
        # Ширина столбцов
        if column_index == 0:
            cell.set_width(0.006)
        else:
            cell.set_width(0.0011)
        
        # Раскраска дублей
        if dups is not None:
            if row_index > 0 and column_index == 0 and cell.get_text().get_text() in dups:
                cell.set_facecolor('#FFC7CE')
                cell.set_text_props(color='#9C0006')
                
        # Цветвоая шкала
        if row_index > 0 and column_index == index_col_to_colorize:
            cell.set_facecolor(colors.pop(0).upper()) 
        
        # Шрифт
        cell.set_text_props(font=fpath, fontsize=28)
    fig.savefig(f'{file_name}', bbox_inches='tight')
    #plt.show()
    plt.close()

In [252]:
make_requests_word('files/ten_city/requests_text_ten_city_230504_0256.xlsx', "", '#249FD8', '#D2DDF1', 
                   size_x=6.3*4, size_y=3.94*4, cnt=100, font_size=28, img_w=16.5, img_h=10, blank=True, word_sample='files/report_ten_city.docx')

files/ten_city/requests_text_ten_city_230504_0256_report_230504_0308.docx


 21%|█████████████████                                                                | 21/100 [01:12<06:17,  4.77s/it]C:\Users\Mamed\AppData\Local\Temp\ipykernel_22084\314543690.py:80: UserWarning: Glyph 1179 (\N{CYRILLIC SMALL LETTER KA WITH DESCENDER}) missing from current font.
  fig.savefig(f'{file_name}', bbox_inches='tight')
 24%|███████████████████▍                                                             | 24/100 [01:30<07:12,  5.69s/it]C:\Users\Mamed\AppData\Local\Temp\ipykernel_22084\314543690.py:80: UserWarning: Glyph 1241 (\N{CYRILLIC SMALL LETTER SCHWA}) missing from current font.
  fig.savefig(f'{file_name}', bbox_inches='tight')
 26%|█████████████████████                                                            | 26/100 [01:47<08:45,  7.10s/it]C:\Users\Mamed\AppData\Local\Temp\ipykernel_22084\314543690.py:80: UserWarning: Glyph 1179 (\N{CYRILLIC SMALL LETTER KA WITH DESCENDER}) missing from current font.
  fig.savefig(f'{file_name}', bbox_inches='tight')
 52%|█████